## Network conversion and weight/activation quantization

This script will load an originally network prototxt file as well as pre trained weights and produces a network file whoch can be loaded by NullHop and run on FPGA

Example case **VGG16:**

network protoxt file can be found either in caffe_lp/examples/low_precision/imagenet/models or here:

https://gist.github.com/ksimonyan/211839e770f7b538e2d8#file-readme-md

caffemodel (original pre trained weights)

http://www.robots.ox.ac.uk/%7Evgg/software/very_deep/caffe/VGG_ILSVRC_16_layers.caffemodel




In [1]:
import caffe
import numpy as np
import matplotlib.pyplot as plt
from caffe.quantization.net_descriptor import net_prototxt
from caffe.quantization.qmf_check import distribute_bits
from caffe.quantization.convert_weights import convert_weights
import os
import sys

%matplotlib inline

# initialize classes
d = distribute_bits()
n = net_prototxt()
c = convert_weights()

In [2]:
'''
The information flow is as follow:
1) Create a folder for a given network you want to quantize/fine tune
   in caffe_lp/examples/low_precision/quantization/network
2) Download and copy .caffemodel file into weight directory (e.g. HDD or wherever you want)
   The copied original file is used to convert blobs (weight W and biases b) into low precision blob structure
3) All models, e.g. prototxt files, are stored in caffe_lp/examples/low_precision/imagenet/models
   We start from the deploy files as they are normally released
4) We distribute the available bits (e.g. 16 bit including sign) for each layer separately for weights and activations
5) We extract the network structure from a given net using extract function within net_prototxt
6) We create a new prototxt file based on extracted network layout and layer-wise bit distribution 
   for weights and activations
7) We finetune the model using the reduced bit precision for 1-5 Epochs

Author: Moritz Milde
Date: 03.11.2016
email: mmilde@ini@uzh.ch
'''

net_name = 'VGG16'
n_bits = 16

caffe_root = '/home/moritz/Repositories/caffe_lp/'
weight_dir = '/media/moritz/Data/ILSVRC2015/pre_trained/'
model_dir = 'examples/low_precision/imagenet/models/'
script_dir = caffe_root + 'examples/low_precision/imagenet/'
layer_dir = 'examples/create_prototxt/layers/'
save_dir = caffe_root + 'examples/low_precision/imagenet/models/'




In [3]:
# converting pre trained caffemodel to low precision blob architecture
c.convert_weights(net_name, save_name=None, 
                   caffe_root=caffe_root, model_dir=model_dir, weight_dir=weight_dir, debug=False)

File already downloaded
Doing forward pass for low precision network
Done.
Saving done caffemodel to /media/moritz/Data/ILSVRC2015/pre_trained/VGG16/HP_VGG16_v2.caffemodel


In [3]:
# We first have to estimate the bit distribution for weights and activations
# since the network will be constructed based on this estimate
# Make sure you downloaded the pre_trained weights from the above link, if VGG16/GoogLeNet, or the respective source
# The naming convention is: NetworkName_deploy.prototxt (for test/one-time rounding)
#                           NetworkName_train.prototxt (for finetune/re-train)
bit_w, net = d.weights(net_name=net_name, n_bits=n_bits, load_mode='high_precision', threshold=0.0,
                       caffe_root=caffe_root, model_dir=model_dir, weight_dir=weight_dir, debug=True)
bit_a, net = d.activation(net_name=net_name, n_bits=n_bits, load_mode='high_precision', threshold=0.0,
                          caffe_root=caffe_root, model_dir=model_dir, weight_dir=weight_dir, debug=True)



Checking if network was already simulated... 
No. Doing forward pass
Forward pass done
Starting extracting weight distribution layer-wise
-------------------
Layer conv1_1: Max: 2.06404
Layer conv1_1: Min: -0.0158289
conv1_1
Before optimaization:
Number of integer bits: 2 
Number of fractional bits: 14
After optimaization:
Number of integer bits: 3 
Number of fractional bits: 13
Done: conv1_1
-------------------
Layer conv1_2: Max: 0.905218
Layer conv1_2: Min: -1.02715
conv1_2
Before optimaization:
Number of integer bits: 0 
Number of fractional bits: 16
After optimaization:
Number of integer bits: 1 
Number of fractional bits: 15
Done: conv1_2
-------------------
Layer conv2_1: Max: 0.365474
Layer conv2_1: Min: -0.179221
conv2_1
Before optimaization:
Number of integer bits: 0 
Number of fractional bits: 16
After optimaization:
Number of integer bits: 1 
Number of fractional bits: 15
Done: conv2_1
-------------------
Layer conv2_2: Max: 0.633758
Layer conv2_2: Min: -0.595335
conv2_2
Be

In [4]:
# By default the functions try to locate models within the directory in (3)!!!!
# Make sure the desired prototxt file to extract the net structure from is in the respective directory
print 'Extracting network structure'
net_layout = n.extract(net_name=net_name, mode='train', model=net, 
                       caffe_root=caffe_root, weight_dir=weight_dir, debug=False)
print net_layout
print 'Creating new network based on weight/activation distribution'
n.create(net_name=net_name, net_descriptor=net_layout, 
         bit_distribution_weights=bit_w, bit_distribution_act=bit_a, scale=True,
         init_method='xavier', lp=True, deploy=False, visualize=False, round_bias='false',
         caffe_root=caffe_root, model_dir=model_dir, layer_dir=layer_dir, save_dir=save_dir, debug=False)


Extracting network structure
['64C3S1p1', 'A', 'ReLU', '64C3S1p1', 'A', 'ReLU', '2P2', '128C3S1p1', 'A', 'ReLU', '128C3S1p1', 'A', 'ReLU', '2P2', '256C3S1p1', 'A', 'ReLU', '256C3S1p1', 'A', 'ReLU', '256C3S1p1', 'A', 'ReLU', '2P2', '512C3S1p1', 'A', 'ReLU', '512C3S1p1', 'A', 'ReLU', '512C3S1p1', 'A', 'ReLU', '2P2', '512C3S1p1', 'A', 'ReLU', '512C3S1p1', 'A', 'ReLU', '512C3S1p1', 'A', 'ReLU', '2P2', '4096F', 'A', 'ReLU', 'D5', '4096F', 'A', 'ReLU', 'D5', '1000F', 'Accuracy', 'loss']
Creating new network based on weight/activation distribution


# Finetune the network
You have to change the lp_solver_GPU0_finetune.prototxt
located in $caffe_root/examples/low_precision/imagenet/solver
to load the netowrk you just created LP_{net_name}_{n_bits}_bits_train.prototxt 
and the converted HP_{net_name}_v2.caffemodel
HAPPY FINETUNING


!!!! Please check data source and mean file path in the prototxt to match your default locations !!!
!!!! 

In [ ]:
cd caffe_root/examples/low_precision/imagenet
gedit finetune.sh
gedit solver/lp_solverGPU0_finetune.prototxt
./finetune.sh